## 1. create high-res base image 

In [1]:
import numpy as np 
import zarr
from dask.distributed import Client 
from dask import array as da 

In [2]:
c = Client(n_workers=4, threads_per_worker=2)

In [3]:
c

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 31.18 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46529,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 31.18 GiB
Comm: tcp://127.0.0.1:41307,Total threads: 2
Dashboard: http://127.0.0.1:40911/status,Memory: 7.80 GiB
Nanny: tcp://127.0.0.1:34263,


In [4]:
# define the highest resolution grid 
highest_res = np.array((512,512,512), dtype=int)
chunks = (64, 64, 64)
grid_dims = np.array(chunks, dtype=int)

# virtual pyramid settings
refine_factor = np.array((2,2,2), dtype=int)

In [5]:
zg = zarr.group('./zarr-test-image-pyramid.zarr', overwrite=True)
field1 = zg.create_group('field1', overwrite=True)

In [6]:
# initialize base high-res level (level 0)
lev0 = da.random.random(tuple(highest_res), chunks=chunks) 
field1.empty(0, shape=highest_res, chunks=chunks)

<zarr.core.Array '/field1/0' (512, 512, 512) float64>

In [7]:
field1.info

Name,/field1
Type,zarr.hierarchy.Group
Read-only,False
Store type,zarr.storage.DirectoryStore
No. members,1
No. arrays,1
No. groups,0
Arrays,0


In [8]:
lev0

dask.array<random_sample, shape=(512, 512, 512), dtype=float64, chunksize=(64, 64, 64), chunktype=numpy.ndarray>

In [9]:
# write to disk
da.to_zarr(lev0, field1["0"])

## 2. logic for downsampling 


Given a single pixel at a given pyramid level, find and average a finer level

In [10]:
def get_fine_ijk(ijk_coarse, level_coarse, level_fine, refine_factor):
    ndim = len(ijk_coarse)
    d_level = level_coarse - level_fine
    ijk_0 = np.array([ijk_coarse[idim] * refine_factor[idim]**d_level for idim in range(ndim)], dtype=int)
    return ijk_0

In [11]:
def find_lev_0_average(ijk_coarse, level_coarse, level_fine, refine_factor, fine_array, offset=None):
    
    ijk = ijk_coarse 
    d_level = level_coarse - level_fine
    
    # get the level 0 index start    
    ijk_0 = get_fine_ijk(ijk, level_coarse, level_fine, refine_factor)
    if offset is None:
        offset = np.zeros(ijk_0.shape,dtype=int)
    ijk_0 = ijk_0 - offset 
    
    # number of fine level pixels covered by one pixel at current level
    lev0_Npixels = np.array(refine_factor**d_level, dtype=int)

    # get end index
    ijk_1 = ijk_0 + lev0_Npixels
    
    val = np.mean(fine_array[ijk_0[0]:ijk_1[0], ijk_0[1]:ijk_1[1], ijk_0[2]:ijk_1[2]])
    return val 

In [12]:
get_fine_ijk((3,1,4), 1, 0, refine_factor), get_fine_ijk((3,1,4), 2, 0, refine_factor)

(array([6, 2, 8]), array([12,  4, 16]))

In [13]:

level_coarse = 2
level_fine = 0 



field1["0"]


<zarr.core.Array '/field1/0' (512, 512, 512) float64>

In [14]:
find_lev_0_average((10,11,12), 2, 0, refine_factor, field1['0'])

0.5151865239295507

In [15]:
def get_level_shape(level_coarse, level_0_res, refine_factor):
    d_level = level_coarse - 0
    return level_0_res // refine_factor**d_level

In [16]:
get_level_shape(3, highest_res, refine_factor)

array([64, 64, 64])

In [17]:
def get_global_start_index(chunk_linear_index, chunks):    
    n_chunks_by_dim = [len(ch) for ch in chunks]
    chunk_index = np.unravel_index(chunk_linear_index, n_chunks_by_dim)    
    ndims = len(chunks)
    si = []
    ei = []
    for idim in range(ndims):
        dim_chunks = np.array(chunks[idim], dtype=int)
        
        covered_chunks = dim_chunks[0:chunk_index[idim]]                
        si.append(np.sum(covered_chunks).astype(int))
        ei.append(si[-1]+chunks[idim][chunk_index[idim]])

    si = np.array(si, dtype=int)
    ei = np.array(ei, dtype=int)
    return si, ei

In [18]:
level = 1
lev_shape = get_level_shape(level, highest_res, refine_factor)
da_lev = da.empty(lev_shape,chunks=chunks)

In [19]:
ichunk = 1 

# current level start/end
si, ei = get_global_start_index(ichunk, da_lev.chunks)

si0 = get_fine_ijk(si, level, 0, refine_factor)
ei0 = get_fine_ijk(ei, level, 0, refine_factor)

print(si0)
print(ei0)
# read in the level 0 range covered by this chunk

covered_vals = field1['0'][si0[0]:ei0[0], si0[1]:ei0[1], si0[2]:ei0[2]]

find_lev_0_average(si, level, 0, refine_factor, covered_vals, offset=si0)

# print(si,ei)
for i0 in range(si[0], ei[1]):
    for i1 in range(si[1], ei[1]):
        for i2 in range(si[2], ei[2]):
            find_lev_0_average(si, level, 0, refine_factor, covered_vals, offset=si0)

[  0   0 128]
[128 128 256]


In [20]:
si

array([ 0,  0, 64])

In [36]:
import numba 


# @numba.jit()
def coarsen(si, ei, output_array, level, fine_level, refine_factor, covered_vals, offset):


    d_level = level_coarse - level_fine
    ndim = len(si)
    
    lev0_Npixels_0 = refine_factor[0]**d_level
    lev0_Npixels_1 = refine_factor[1]**d_level
    lev0_Npixels_2 = refine_factor[2]**d_level

    # output = np.zeros(output_shape, dtype=np.float64)
    # outputs = []
    for i0_coarse in range(si[0], ei[0]):
        i0_fine_0 = i0_coarse * refine_factor[0]  ** d_level + offset[0]        
        i0_fine_1 = i0_fine_0 + lev0_Npixels_0
        for i1_coarse in range(si[1], ei[1]):
            i1_fine_0 = i1_coarse * refine_factor[1] ** d_level + offset[1]
            i1_fine_1 = i1_fine_0 + lev0_Npixels_1
            for i2_coarse in range(si[2], ei[2]):
                i2_fine_0 = i2_coarse * refine_factor[2] ** d_level  + offset[2]                         
                i2_fine_1 = i2_fine_0 + lev0_Npixels_2
                
                val = 0.0
                nvals = 0.0
                print(i2_fine_0, i2_fine_1)
                for i0 in range(i0_fine_0, i0_fine_1):
                    for i1 in range(i1_fine_0, i1_fine_1):
                        for i2 in range(i2_fine_0, i2_fine_1):
                            print(i0_coarse, i1_coarse, i2_coarse)
                            print(i0,i1,i2)
                            val += covered_vals[i0, i1, i2]
                            nvals += 1.0
                val = float(val / nvals)
                output_array[i0_coarse,i1_coarse, i2_coarse] = val 

## wtf why is it crashing
    

In [37]:
import numpy as np 
import numba 

@numba.jit()
def array_copy(x1, x2):
    for ix in range(0, x1.shape[0]):
        x2[ix] = x1[ix]



In [38]:
x1 = np.linspace(0,10,100) 
x2 = np.empty(x1.shape)

In [39]:
array_copy(x1,x2)

In [40]:
si, ei, covered_vals.shape, si0

(array([ 0,  0, 64]),
 array([ 64,  64, 128]),
 (128, 128, 128),
 array([  0,   0, 128]))

In [41]:
covered_vals.min(), covered_vals.max()

(9.950673569569801e-07, 0.9999996929229337)

In [42]:
offsets = -1 * si0
offsets

array([   0,    0, -128])

In [43]:
output_array = np.zeros(ei-si)
coarsen(si, ei, output_array, level, 0, refine_factor, covered_vals, offsets)

128 132
0 0 64
0 0 128


IndexError: index 128 is out of bounds for axis 2 with size 128

In [24]:
output_array

array([[[0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        ...,
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]],

       [[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        ...,
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]],

       [[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        ...,
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]],

       ...,

       [[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5

## 3. Using the downsampler

In [14]:
import numba 

In [ ]:
@jit
def go_fast(a): # Function is compiled to machine code when called the first time
    trace = 0.0
    for i in range(a.shape[0]):   # Numba likes loops
        trace += np.tanh(a[i, i]) # Numba likes NumPy functions
    return a + trace              # Numba likes NumPy broadcasting

In [14]:
# target array 
lev = 1 
lev_shape = get_level_shape(lev, highest_res, refine_factor)
lev_array = da.empty(lev_shape, chunks=chunks)
lev_array

dask.array<empty_like, shape=(256, 256, 256), dtype=float64, chunksize=(64, 64, 64), chunktype=numpy.ndarray>

In [15]:
for linear_ijk in range(10):
    ijk = np.unravel_index(linear_ijk, lev_array.shape, order='F')
    lev_array[ijk] = find_lev_0_average(ijk, lev, 0, refine_factor, field1['0'])

In [17]:
lev_array[0,0,0].compute()

np.float64(0.45460859291196165)

In [38]:
%%time
find_lev_0_average((0,0,0), lev, 0, refine_factor, field1['0'])

CPU times: user 3.9 ms, sys: 687 μs, total: 4.59 ms
Wall time: 3.48 ms


np.float64(0.6399256447305695)

In [ ]:
# initialize the levels 
from dask import array as da 

n_levels=5
da_levels = {}
for lev in range(0, n_levels):
    lev_shape = highest_res / (refine_factor**lev)    
    if np.any(lev_shape / grid_dims < 1): 
        break
    max_levels = lev    
    da_levels[lev] = da.random.random(tuple(lev_shape), 
                                         chunks=chunks)        
    field1.empty(lev, shape=lev_shape, chunks=grid_dims)

print(max_levels)
    

In [268]:
import numpy as np 
import yt 
import zarr
import dask

# define the highest resolution grid 
highest_res = np.array((512,512,512), dtype=int)
chunks = (64, 64, 64)
grid_dims = np.array(chunks, dtype=int)

# virtual pyramid settings
refine_factor = np.array((2,2,2), dtype=int)

# spatial extent of the whole domain
domain_le = np.array([0., 0., 0.])
domain_re = np.array([1., 1., 1.])

In [269]:
zg = zarr.group('./zarr-test-image-pyramid.zarr', overwrite=True)
field1 = zg.create_group('field1', overwrite=True)

In [270]:
# initialize the levels 
from dask import array as da 

n_levels=5
da_levels = {}
for lev in range(0, n_levels):
    lev_shape = highest_res / (refine_factor**lev)    
    if np.any(lev_shape / grid_dims < 1): 
        break
    max_levels = lev    
    da_levels[lev] = da.random.random(tuple(lev_shape), 
                                         chunks=chunks)        
    field1.empty(lev, shape=lev_shape, chunks=grid_dims)

print(max_levels)
    

3


In [271]:
def get_global_start_index(chunk_linear_index, chunks):    
    n_chunks_by_dim = [len(ch) for ch in chunks]
    chunk_index = np.unravel_index(chunk_linear_index, n_chunks_by_dim)    
    ndims = len(chunks)
    si = []
    ei = []
    for idim in range(ndims):
        dim_chunks = np.array(chunks[idim], dtype=int)
        
        covered_chunks = dim_chunks[0:chunk_index[idim]]                
        si.append(np.sum(covered_chunks).astype(int))
        ei.append(si[-1]+chunks[idim][chunk_index[idim]])

    si = np.array(si, dtype=int)
    ei = np.array(ei, dtype=int)
    return si, ei

In [274]:
def find_lev_0_average(ijk, pyramid_level):
    # get the level 0 index start
    ndim=len(ijk)
    ijk_0 = np.array([ijk[idim] * refine_factor[idim]**(pyramid_level-0) for idim in range(ndim)], dtype=int)
    
    # number of level 0 pixels covered by one pixel at current level
    lev0_Npixels = np.array(refine_factor**(pyramid_level-0), dtype=int)

    # get end index
    ijk_1 = ijk_0 + lev0_Npixels

    val = np.mean(da_levels[0][ijk_0[0]:ijk_1[0], ijk_0[1]:ijk_1[1], ijk_0[2]:ijk_1[2]])
    return val 
                    

np.float64(0.5015647969711297)

In [273]:
pyramid_level = 2
da_lev = da_levels[pyramid_level]

n_chunks = np.prod([len(ch) for ch in da_lev.chunks])
for ichunk in range(n_chunks):

    # start/end indices at this level and chunk
    si, ei = get_global_start_index(ichunk, da_lev.chunks)

    # for every pixel, find average of level 0 covered pixels 
    
    
    # starting indices at level 0
    si_lev0 = np.array([si[idim] * refine_factor[idim]**(pyramid_level-0) for idim in range(3)], dtype=int)
    
    # number of level 0 pixels covered by one pixel at current level
    lev0_Npixels = refine_factor**(pyramid_level-0)
    
    ei_lev0 = si_lev0+lev0_Npixels*(ei - si)

    print(f"chunk {ichunk}")
    print(si, si_lev0)
    print(ei, ei_lev0) 

    


chunk 0
[0 0 0] [0 0 0]
[64 64 64] [256 256 256]
chunk 1
[ 0  0 64] [  0   0 256]
[ 64  64 128] [256 256 512]
chunk 2
[ 0 64  0] [  0 256   0]
[ 64 128  64] [256 512 256]
chunk 3
[ 0 64 64] [  0 256 256]
[ 64 128 128] [256 512 512]
chunk 4
[64  0  0] [256   0   0]
[128  64  64] [512 256 256]
chunk 5
[64  0 64] [256   0 256]
[128  64 128] [512 256 512]
chunk 6
[64 64  0] [256 256   0]
[128 128  64] [512 512 256]
chunk 7
[64 64 64] [256 256 256]
[128 128 128] [512 512 512]


np.int64(8)

In [103]:
da_lev.blocks[np.unravel_index(0, n_chunks)]

dask.array<blocks, shape=(64, 64, 64), dtype=float64, chunksize=(64, 64, 64), chunktype=numpy.ndarray>

In [106]:
block = da_lev.blocks[np.unravel_index(0, n_chunks)]
block.

(64, 64, 64)

In [ ]:
def write_level(da_lev, lev):

    n_chunks = [len(ch) for ch in da_lev.chunks]
    for ch_x in range(n_chunks[0]):
        ix0 = np.prod(da_lev.chunks[0][:ch_x])
        ix1 = ix0 + da_lev.chunks[0][ch_x]                      
        for ch_y in range(n_chunks[0]):
            for ch_z in range(n_chunks[0]):
                
    da_lev

In [63]:
da.random.random((10,10,10), chunks=(2,2,2))

dask.array<random_sample, shape=(10, 10, 10), dtype=float64, chunksize=(2, 2, 2), chunktype=numpy.ndarray>

In [109]:
f1_0 = da.from_zarr(field1['0'])
f1_0

dask.array<from-zarr, shape=(256, 256, 256), dtype=float64, chunksize=(64, 64, 64), chunktype=numpy.ndarray>

In [48]:
blocks = np.array(field1['0'].shape) / np.array(field1['0'].chunks)
blocks

array([4., 4., 4.])

In [ ]:
for block in blocks.ravel():
    print(block)

In [36]:
64*64*64

262144

In [ ]:
for field in ('field1', 'field2'):    
    zg.array(field, np.random.random(highest_res), chunks=tuple(grid_dims))

In [ ]:
fld3 = np.zeros(highest_res)
fld3[highest_res[0]//3:,highest_res[1]//3:,highest_res[2]//3:] = 1.0 
zg.array('field3', fld3, chunks=tuple(grid_dims))

In [ ]:
pyramid_level = max_level - level
    
if pyramid_level == 0: 
    # just read direct from zarr
    vals = zg[field][si[0]:ei[0], 
                     si[1]:ei[1], 
                     si[2]:ei[2]]
    return vals     

# level > 0: average all the covered pixels 

# starting indices at current level 
xyz_i = [np.arange(si[idim], ei[idim]) for idim in range(3)]

# starting indices at level 0
xyz_i_lev0 = [xyz_i[idim] * refine_factor[idim]**(pyramid_level-0) for idim in range(3)]
# number of level 0 pixels covered by one pixel at current level
lev0_Npixels = refine_factor**(pyramid_level-0)
# starting level 0 index permuations 
xi0, yi0, zi0 = np.meshgrid(*xyz_i_lev0, indexing='ij')
xi1d = xi0.ravel()
yi1d = yi0.ravel()
zi1d = zi0.ravel()                        

# number of level 0 pixels covered by one pixel at current level
lev0_Npixels = refine_factor**(pyramid_level-0)
# print(level, lev0_Npixels, si, ei)

# find average of covered level 0 pixels -- this is a bottleneck
im0 = np.zeros(xi1d.shape)
N_pixels = 0
for offset_x in range(0, lev0_Npixels[0]):
    for offset_y in range(0, lev0_Npixels[1]):
        for offset_z in range(0, lev0_Npixels[2]):
            # print(offset_x, offset_y, offset_z)
            im0 += zg[field][xi1d+offset_x, yi1d+offset_y, zi1d+offset_z]
            N_pixels += 1

im0 /= N_pixels
im0 = im0.reshape(ei - si)

Init signature: np.ndindex(*shape)
Docstring:     
An N-dimensional iterator object to index arrays.

Given the shape of an array, an `ndindex` instance iterates over
the N-dimensional index of the array. At each iteration a tuple
of indices is returned, the last dimension is iterated over first.

Parameters
----------
shape : ints, or a single tuple of ints
    The size of each dimension of the array can be passed as
    individual parameters or as the elements of a tuple.

See Also
--------
ndenumerate, flatiter

Examples
--------
>>> import numpy as np

Dimensions as individual arguments

>>> for index in np.ndindex(3, 2, 1):
...     print(index)
(0, 0, 0)
(0, 1, 0)
(1, 0, 0)
(1, 1, 0)
(2, 0, 0)
(2, 1, 0)

Same dimensions - but in a tuple ``(3, 2, 1)``

>>> for index in np.ndindex((3, 2, 1)):
...     print(index)
(0, 0, 0)
(0, 1, 0)
(1, 0, 0)
(1, 1, 0)
(2, 0, 0)
(2, 1, 0)
File:           ~/.pyenv/versions/3.11.7/envs/zarr_explorations/lib/python3.11/site-packages/numpy/__init__.py
T

In [250]:
from dask import array as da 
import numpy as np 

x = da.empty((100,100,100),chunks=(50,50,50), dtype=np.float64)
x.chunks

((50, 50), (50, 50), (50, 50))

In [251]:
x

dask.array<empty_like, shape=(100, 100, 100), dtype=float64, chunksize=(50, 50, 50), chunktype=numpy.ndarray>

In [256]:
x = da.arange(1000, chunks=(100,))


def func(block_info=None):
    print(block_info)
    loc = block_info[None]['array-location'][0]
    return np.arange(loc[0], loc[1])

da.map_blocks(func, chunks=((4, 4),), dtype=np.float64).compute()

None
[]


array([0, 1, 2, 3, 4, 5, 6, 7])

In [252]:
def average_level(x, block_info=None):
    print("hello")
    print(block_info)
    
    # def func(block_info=None):

    # loc = block_info[None]['array-location'][0]

    # return np.arange(loc[0], loc[1])

In [253]:
_ = da.map_blocks(average_level, x).compute()

hello
None
hello
None
hello
None
hello
[]


IndexError: tuple index out of range

In [241]:
x.numblocks

(10, 10, 10)

In [197]:
x.chunks[0][0]

50

In [211]:
block_index = (0,2,1) 
si_0 = int(np.sum(x.chunks[0][0:block_index[0]]))
si_1 = int(np.sum(x.chunks[1][0:block_index[1]]))
si_2 = int(np.sum(x.chunks[2][0:block_index[2]]))
print(si_0, si_1, si_2)

0 100 50


In [208]:
ei_0 = si_0 + x.chunks[0][block_index[0]]
ei_1 = si_1 + x.chunks[1][block_index[1]]
ei_2 = si_2 + x.chunks[2][block_index[2]]
print(ei_0, ei_1, ei_2)

50 150 100


In [ ]:

def get_global_start_index(chunk_linear_index, da_array):  
    chunks = da_array.chunks
    n_chunks_by_dim = [len(ch) for ch in chunks]
    chunk_index = np.unravel_index(chunk_linear_index, n_chunks_by_dim)    
    ndims = len(chunks)
    si = []
    ei = []
    for idim in range(ndims):
        dim_chunks = np.array(chunks[idim], dtype=int)
        
        covered_chunks = dim_chunks[0:chunk_index[idim]]
        if len(covered_chunks) == 0: 
            covered_chunks = 0
        
        si.append(int(np.prod(covered_chunks)))
        ei.append(si[-1]+chunks[idim][chunk_index[idim]])
    return si, ei


In [215]:
block0 = x.blocks[0]
block0

dask.array<blocks, shape=(50, 500, 500), dtype=float64, chunksize=(50, 50, 50), chunktype=numpy.ndarray>

In [222]:
x.blocks[0,0,0].block_id

AttributeError: 'Array' object has no attribute 'block_id'

In [223]:
x.block_id

AttributeError: 'Array' object has no attribute 'block_id'